In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [2]:
df=pd.read_csv("creditcard.csv")   
df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284807 entries, 0 to 284806
Data columns (total 31 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Time    284807 non-null  float64
 1   V1      284807 non-null  float64
 2   V2      284807 non-null  float64
 3   V3      284807 non-null  float64
 4   V4      284807 non-null  float64
 5   V5      284807 non-null  float64
 6   V6      284807 non-null  float64
 7   V7      284807 non-null  float64
 8   V8      284807 non-null  float64
 9   V9      284807 non-null  float64
 10  V10     284807 non-null  float64
 11  V11     284807 non-null  float64
 12  V12     284807 non-null  float64
 13  V13     284807 non-null  float64
 14  V14     284807 non-null  float64
 15  V15     284807 non-null  float64
 16  V16     284807 non-null  float64
 17  V17     284807 non-null  float64
 18  V18     284807 non-null  float64
 19  V19     284807 non-null  float64
 20  V20     284807 non-null  float64
 21  V21     28

In [4]:
df.describe()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
count,284807.000000,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,...,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,284807.000000,284807.000000
mean,94813.859575,1.168375e-15,3.416908e-16,-1.379537e-15,2.074095e-15,9.604066e-16,1.487313e-15,-5.556467e-16,1.213481e-16,-2.406331e-15,...,1.654067e-16,-3.568593e-16,2.578648e-16,4.473266e-15,5.340915e-16,1.683437e-15,-3.660091e-16,-1.227390e-16,88.349619,0.001727
std,47488.145955,1.958696e+00,1.651309e+00,1.516255e+00,1.415869e+00,1.380247e+00,1.332271e+00,1.237094e+00,1.194353e+00,1.098632e+00,...,7.345240e-01,7.257016e-01,6.244603e-01,6.056471e-01,5.212781e-01,4.822270e-01,4.036325e-01,3.300833e-01,250.120109,0.041527
min,0.000000,-5.640751e+01,-7.271573e+01,-4.832559e+01,-5.683171e+00,-1.137433e+02,-2.616051e+01,-4.355724e+01,-7.321672e+01,-1.343407e+01,...,-3.483038e+01,-1.093314e+01,-4.480774e+01,-2.836627e+00,-1.029540e+01,-2.604551e+00,-2.256568e+01,-1.543008e+01,0.000000,0.000000
25%,54201.500000,-9.203734e-01,-5.985499e-01,-8.903648e-01,-8.486401e-01,-6.915971e-01,-7.682956e-01,-5.540759e-01,-2.086297e-01,-6.430976e-01,...,-2.283949e-01,-5.423504e-01,-1.618463e-01,-3.545861e-01,-3.171451e-01,-3.269839e-01,-7.083953e-02,-5.295979e-02,5.600000,0.000000
50%,84692.000000,1.810880e-02,6.548556e-02,1.798463e-01,-1.984653e-02,-5.433583e-02,-2.741871e-01,4.010308e-02,2.235804e-02,-5.142873e-02,...,-2.945017e-02,6.781943e-03,-1.119293e-02,4.097606e-02,1.659350e-02,-5.213911e-02,1.342146e-03,1.124383e-02,22.000000,0.000000
75%,139320.500000,1.315642e+00,8.037239e-01,1.027196e+00,7.433413e-01,6.119264e-01,3.985649e-01,5.704361e-01,3.273459e-01,5.971390e-01,...,1.863772e-01,5.285536e-01,1.476421e-01,4.395266e-01,3.507156e-01,2.409522e-01,9.104512e-02,7.827995e-02,77.165000,0.000000
max,172792.000000,2.454930e+00,2.205773e+01,9.382558e+00,1.687534e+01,3.480167e+01,7.330163e+01,1.205895e+02,2.000721e+01,1.559499e+01,...,2.720284e+01,1.050309e+01,2.252841e+01,4.584549e+00,7.519589e+00,3.517346e+00,3.161220e+01,3.384781e+01,25691.160000,1.000000


In [5]:
df['Class'].value_counts()

0    284315
1       492
Name: Class, dtype: int64

In [6]:
fraud=df[df["Class"]==1]
not_fraud=df[df["Class"]==0]

In [11]:
fraud.Amount.describe()

count     492.000000
mean      122.211321
std       256.683288
min         0.000000
25%         1.000000
50%         9.250000
75%       105.890000
max      2125.870000
Name: Amount, dtype: float64

In [10]:
not_fraud.Amount.describe()

count    284315.000000
mean         88.291022
std         250.105092
min           0.000000
25%           5.650000
50%          22.000000
75%          77.050000
max       25691.160000
Name: Amount, dtype: float64

In [13]:
not_fraud=not_fraud.sample(n=492)
not_fraud

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
172536,121137.0,-1.195380,-2.150306,-3.223257,0.673080,1.517977,-1.356786,3.001859,-0.769473,-0.473996,...,0.897980,1.638540,1.574784,-0.568442,-0.626965,-0.437315,0.093052,0.149664,632.35,0
900,678.0,-0.443821,0.070782,2.038546,-1.515417,-0.133101,-0.330720,0.482787,-0.904148,-0.558080,...,0.240493,1.104593,-0.324978,-0.051664,-0.004635,-0.249176,-0.922113,-0.599479,15.80,0
240004,150381.0,-1.702203,1.659175,1.074689,0.893458,-0.197329,0.779488,-0.213446,1.212691,-0.519708,...,-0.452173,-1.177353,-0.065817,0.657711,0.485510,-0.827588,0.237751,0.045058,16.90,0
31117,36217.0,-0.054071,1.439225,-0.677409,0.643573,0.790326,-0.377560,0.669921,0.208617,-0.706191,...,0.091269,0.399674,-0.116907,-0.982002,-0.477254,-0.328598,0.317907,0.155651,1.79,0
161725,114460.0,-2.704615,0.623843,-0.290293,-3.809662,-0.853179,0.273286,-0.457169,1.449531,-0.597001,...,-0.787441,-1.857368,-0.117562,-0.233577,0.687453,-0.215867,0.196488,-0.117228,86.99,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19888,30628.0,1.015427,-0.396231,0.717847,1.360296,-0.735631,0.283377,-0.433917,0.266482,0.926393,...,-0.058798,-0.087225,-0.196481,0.001727,0.630529,-0.302805,0.031088,0.020834,75.37,0
161858,114586.0,2.042226,-0.043226,-1.158795,0.216604,0.151843,-0.684204,0.137653,-0.219814,0.164252,...,-0.247877,-0.570137,0.277046,-0.310760,-0.254743,0.195186,-0.068983,-0.071345,1.98,0
66507,52085.0,0.805035,-0.180480,0.873124,2.402943,-0.317043,0.966447,-0.426676,0.474479,-0.511334,...,0.274742,0.468955,-0.077474,-0.314934,0.140955,0.102413,0.009849,0.030582,122.47,0
212132,138730.0,-3.606073,-2.663018,0.104955,-2.594181,0.584785,-1.410353,1.280764,-1.531407,-1.300163,...,-1.048198,-0.057712,0.824973,0.079249,0.361572,-0.478740,-1.095355,0.418438,108.15,0


In [14]:
palanced_dataset=pd.concat([fraud,not_fraud],axis=0)
palanced_dataset

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
541,406.0,-2.312227,1.951992,-1.609851,3.997906,-0.522188,-1.426545,-2.537387,1.391657,-2.770089,...,0.517232,-0.035049,-0.465211,0.320198,0.044519,0.177840,0.261145,-0.143276,0.00,1
623,472.0,-3.043541,-3.157307,1.088463,2.288644,1.359805,-1.064823,0.325574,-0.067794,-0.270953,...,0.661696,0.435477,1.375966,-0.293803,0.279798,-0.145362,-0.252773,0.035764,529.00,1
4920,4462.0,-2.303350,1.759247,-0.359745,2.330243,-0.821628,-0.075788,0.562320,-0.399147,-0.238253,...,-0.294166,-0.932391,0.172726,-0.087330,-0.156114,-0.542628,0.039566,-0.153029,239.93,1
6108,6986.0,-4.397974,1.358367,-2.592844,2.679787,-1.128131,-1.706536,-3.496197,-0.248778,-0.247768,...,0.573574,0.176968,-0.436207,-0.053502,0.252405,-0.657488,-0.827136,0.849573,59.00,1
6329,7519.0,1.234235,3.019740,-4.304597,4.732795,3.624201,-1.357746,1.713445,-0.496358,-1.282858,...,-0.379068,-0.704181,-0.656805,-1.632653,1.488901,0.566797,-0.010016,0.146793,1.00,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19888,30628.0,1.015427,-0.396231,0.717847,1.360296,-0.735631,0.283377,-0.433917,0.266482,0.926393,...,-0.058798,-0.087225,-0.196481,0.001727,0.630529,-0.302805,0.031088,0.020834,75.37,0
161858,114586.0,2.042226,-0.043226,-1.158795,0.216604,0.151843,-0.684204,0.137653,-0.219814,0.164252,...,-0.247877,-0.570137,0.277046,-0.310760,-0.254743,0.195186,-0.068983,-0.071345,1.98,0
66507,52085.0,0.805035,-0.180480,0.873124,2.402943,-0.317043,0.966447,-0.426676,0.474479,-0.511334,...,0.274742,0.468955,-0.077474,-0.314934,0.140955,0.102413,0.009849,0.030582,122.47,0
212132,138730.0,-3.606073,-2.663018,0.104955,-2.594181,0.584785,-1.410353,1.280764,-1.531407,-1.300163,...,-1.048198,-0.057712,0.824973,0.079249,0.361572,-0.478740,-1.095355,0.418438,108.15,0


In [15]:
palanced_dataset["Class"].value_counts()

1    492
0    492
Name: Class, dtype: int64

In [16]:
x=palanced_dataset.drop("Class",axis='columns')
y=palanced_dataset["Class"]

In [17]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,stratify=y,random_state=2)

In [18]:
logs_model=LogisticRegression()
logs_model.fit(x_train,y_train)

/Users/shubhambhatt/anaconda3/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [19]:
y_predict=logs_model.predict(x_test)

In [20]:
accuracy=accuracy_score(y_test,y_predict)
print("Accuracy= ",int(accuracy*100),'%')

Accuracy=  91 %
